In [1]:
import jieba

In [2]:
from hanziconv import HanziConv

In [3]:
def write_token_to_f(open_file, output_file):
    words = []
    for line in open(open_file):
        # 繁体字转换成简体字
        conv_line = HanziConv.toSimplified(line)
        w = list(jieba.cut(conv_line))
        words += w + ['\n']
    output_file.writelines(' '.join(words))

In [4]:
import os

In [5]:
target_files = os.listdir('zhwiki')

In [6]:
import time

In [7]:
start = time.time()
with open('train_corpus.txt', 'w') as output_f:
    for file in target_files:
        subfiles = os.listdir('zhwiki'+'/'+file)
        num = 0
        for f in subfiles:
            if num > 37: break
            if not f.startswith('wiki'): continue
            write_token_to_f('zhwiki'+'/'+file+'/'+f, output_f)
            num += 1
end = time.time()
print('used time {}'.format(end-start))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_8/f14fxxnn7w13cd0l5x9hl4bm0000gn/T/jieba.cache
Loading model cost 0.880 seconds.
Prefix dict has been built succesfully.


   used time 105.68170404434204


In [7]:
from gensim.models import Word2Vec

In [8]:
from gensim.models.word2vec import LineSentence

In [9]:
model = Word2Vec.load('zhwiki_model')

In [11]:
s = time.time()
model = Word2Vec(LineSentence('train_corpus.txt'), min_count=1)
e = time.time()
print('used time {}'.format(e-s))

   used time 67.79696822166443


In [12]:
model.save('zhwiki_model')

In [10]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def tsne_plot(model):
    '''
    Creates TSNE model and plots it.
    '''
    labels = []
    tokens = []
    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    tsne_model = TSNE(n_components=2, learning_rate=100)
    new_values = tsne_model.fit_transform(tokens)
    
    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(model)